In [1]:
!pip install scipy numpy ipywidgets matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<style>
   .imagecontainer {
        display: flex;
        justify-content: space-around;
        align-items: center;
        gap: 10px;
     }

    img {
        width: 55vw;
        min-width: 330px;
    }
</style>

# Finde ein effizientes Programm um eine Floyd Rose Gitarre zu stimmen

## Einleitung
Es gibt ein Problem beim Stimmen von Floyd Rose Gitarren. Bei diesen Gitarren wird eine Saite zwischen dem Gitarren Kopf und einer freirotierbaren Brücke gespannt. Damit die Brücke nicht einfach umklappt wenn man Saiten einspannt, halten unterhalb des Drehpunkts Federn dagegen.

### Bilder von der Gitarre
<div class="imagecontainer">
    <img  class="img" src="assets/floydrose_back.jpg"></img>
</div>

#### Neutrale Lage
<div class="imagecontainer">
    <img src="assets/floydrose_frontside_neutral.jpg"></img>
    <img src="assets/floydrose_backside_neutral.jpg"></img>
</div>

#### Saiten gestresst
<div class="imagecontainer">
    <img src="assets/floydrose_frontside_strings_stressed.jpg"></img>
    <img src="assets/floydrose_backside_strings_stressed.jpg"></img>
</div>

#### Saiten entspannter
<div class="imagecontainer">
    <img src="assets/floydrose_frontside_strings_released.jpg"></img>
    <img src="assets/floydrose_backside_strings_released.jpg"></img>
</div>

## Problem

Diese Architektur erschwert das Stimmen der Gitarre erheblich. Beim Stimmen erhöht oder verringert man die Spannung einer Saite. Demnach erhöht oder verringert sich die Auslenkung der Federn. Das wiederum verändert die Auslenkung der Brücke und damit wieder die Spannung der anderen Saiten. Das führt dazu, dass die anderen Saiten verstimmt werden, wenn man eine Saite stimmt.


## Ziel

In dieser Arbeit soll ein Weg gefunden werden, alle Saiten in die richtige Stimmung zu bringen, wobei jede Saite nur einmal gestimmt werden muss.

## Methodik
Zunächst stellen wir ein mathematisches Modell auf, welches das Verhalten der Saiten beschreiben soll. Anschließend überprüfen wir das Modell mit einem Experiment.
Dann werden die gewonnenen Erkenntnisse genutzt, um das Ziel in ein mathematisches Problem zu überführen.
Zum Schluss wird die mathematische Lösung als Programm umgesetzt.


## Mathematisches Modell

Da es um das Stimmen der Saite geht, müssen wir zunächst verstehen welche Faktoren einfluss auf die Frequenz der Saite haben.
Die Frequenz einer Saite wird bestimmt durch [3]
$$
    f = \frac{1}{2L} \sqrt{\frac{T}{\mu}}
$$

Wobei $f$ die Frequenz, $L_S$ die Länge, $T$ die Spannung und $\mu$ die lineare Dichte der Saite ist.

$\mu$ ist konstant und Saiten abhängig.

Die Spannung $T$ ist proportional zur Kraft, die an der Saite zieht.[4]
$$
    T = \frac{F}{A}
$$

Wobei $F$ die Kraft ist und $A$ die Querschnittsfläche der Saite.

Die Querschnittsfläche $A$ ist konstant und Saiten abhängig.

Die Kraft $F$ ist die Kraft, mit der die Federn an der Brücke ziehen.

Die Federkraft ist proportional zu Auslenkung der Federn.[5]

$$
    F = L_F \cdot k
$$

$k$ ist die Federkonstante und $L_F$ die Auslenkung der Federn.
$k$ ist konstant und Feder abhängig.

Kombinieren wir nun also diese Gleichungen erhalten wir die Frequenz der Saite in Abhängigkeit der Auslenkung der Federn.

$$
    f = \frac{1}{2L} \sqrt{\frac{L_F \cdot k}{\mu}}
$$

Da die Brücke rotiert, wenn man die Saiten verkürzt ist die Auslenkung der Federn nicht linear zur Verkürzung der Saiten.

Um die Abhängigkeit zwischen Auslenkung der Feder und Länge der Saite zu bestimmen, betrachten wir die Gitarre als Kombination von Punkten.

<img src="assets/math_model.png" />

- Die Punkte $A$, $B$, $C$ sind konstant.
- Der Winkel $\alpha = \sphericalangle{(\vec{u},\vec{v})} $ ist konstant.
- Der Winkel $\beta = \sphericalangle{(\vec{v},\vec{e_x})}$ ist variabel.
- $L_S,L_F$ sind Variable größen.

Gesucht ist die Funktion $L_F(L_S)$.

Die Punkte $A$, $C$ und $D$ bilden ein Dreieck. Der Innenwinkel an Punkt $C$ ist:
$$
\varphi = \pi - \beta
$$

Das Dreieck hat die Saiten Längen $L_S$, $a_x$, $u$ wobei $A = (-a_x, a_y)$, $u = |\vec{u}|$ \
Mit dem Kosinussatz erhalten wir:

$$
    L_S^2 = a_x^2 + u^2 - 2 a_x u \cos(\varphi)
$$

umgestellt nach $\varphi$ erhalten wir:

$$
    \varphi = \cos^{-1}(\frac{a_x^2 + u^2 - L_S^2}{2 a_x u})
$$

Da $\varphi = \pi - \beta$ ist, erhalten wir:

$$
    \beta(L_S) = \pi - \cos^{-1}(\frac{a_x^2 + u^2 - L_S^2}{2 a_x u})
$$

$ v = |\vec v| $

So Erhalten wir den Vektor $ \vec v = v \begin{pmatrix} \cos(\beta(L_S) - \alpha) \\ \sin(\beta(L_S) - \alpha) \end{pmatrix} $

$ L_F = |\vec B - \vec v|$

Daraus folgt:

$$
    L_F(L_S) = \sqrt{(a_x - v \cos(\beta(L_S) - \alpha))^2 + (a_y - v \sin(\beta(L_S) - \alpha))^2}
$$
$$
    L_F(L_S) = \sqrt{v^2 \cos^2(\beta(L_S) - \alpha) + v^2 \sin^2(\beta(L_S) - \alpha) +  a_x^2 + a_y^2 - 2v( \cos(\beta(L_S) - \alpha) a_x + \sin(\beta(L_S) - \alpha) a_y)}
$$
$$
    L_F(L_S) = \sqrt{v^2 + a_x^2 + a_y^2 - 2v( \cos(\beta(L_S) - \alpha) a_x + \sin(\beta(L_S) - \alpha) a_y)}
$$

$$
    L_F(L_S) = \sqrt{v^2 + a_x^2 + a_y^2 - 2v( \cos(\pi - \cos^{-1}(\frac{a_x^2 + u^2 - L_S^2}{2 a_x u}) - \alpha) a_x + \sin(\pi - \cos^{-1}(\frac{a_x^2 + u^2 - L_S^2}{2 a_x u}) - \alpha) a_y)}
$$

Kombinieren wir nun alle Erkenntnisse erhalten wir die Funktion
$$
    f(L_S) = \frac{1}{2L_S} \sqrt{\frac{\sqrt{v^2 + a_x^2 + a_y^2 - 2v( \cos(\pi - \cos^{-1}(\frac{a_x^2 + u^2 - L_S^2}{2 a_x u}) - \alpha) a_x + \sin(\pi - \cos^{-1}(\frac{a_x^2 + u^2 - L_S^2}{2 a_x u}) - \alpha) a_y)} \cdot k}{\mu}}
$$

Diese Funktion sieht so aus:

<img src="assets/f(L_s).png">


### Diskussion
Die Korrelation zwischen Verlängerung der Saite und der Abnahme trifft zu da wir eine Negative Steigung sehen.
Zu Beachten ist, dass nur der mittlere Teil der Funktion relevant ist da die Grenzfälle von $\beta$ durch Begrenzung des Rotationsspiels nicht erreicht werden können. Schätzungsweise bewegt sich Beta zwischen 5° bis 30°.




## Experiment

Um die Theorie zu überprüfen, wird ein Experiment durchgeführt, welches die Frequenzänderung der Saiten misst, wenn eine andere Saite verstimmt wird.

Dazu wird zunächst jede Saite in eine Ausgangsposition gebracht. Eine standard Gitarren Stimmung ist EADGBE.
Da eine Gitarre sich aufgrund von menschlichen Fehler nicht perfekt stimmen lässt, wird die Ausgangsfrequenz der Saiten gemessen und aufgezeichnet.

Dann wird jede Saite um ein beliebiges delta verstimmt. Das Delta wird so gewählt dass die Saite sich deutlich hörbar verstimmt. Auch dieses delta wird gemessen und aufgezeichnet.

Jede Saite wird um 4 Schritte jeweils nach oben und nach unten verstimmt. Für jeden Schritt wird die Frequenz der anderen Saiten gemessen und aufgezeichnet.

Das Ziel des Experiments ist zu beobachten, ob das System
- hinreichend Linear ist (Nach der Theorie nicht zu erwarten)
- elastisch ist

Elastisch ist es, wenn die Frequenz einer Saite wieder in ihren Ausgangszustand zurückkehrt, wenn die Verstimmung aufgehoben wird.
Dazu wird nach allen Verstimmungsschritten verglichen, ob die Frequenz der Saiten wieder in den Ausgangszustand zurückkehrt.

Zum Messen der Frequenz, wird ein Python Programm sein, welches die Frequenz der Saiten mittels Fourier-Transformation berechnet. Die Visualisierungssoftware wird ebenfalls ein Python Programm sein, welches mit matplotlib.pyplot die Graphen anzeigt.

#### Frequenz Messgerät

Gegeben sei eine Liste von Audio-Samples. Diese Liste wird mit Hilfe der Fourier Transformation in den Frequenzbereich transformiert. Die Frequenz mit der höchsten Amplitude wird als die Frequenz der Saite angenommen.


In [2]:
from scipy.fftpack import fft
from scipy.io import wavfile  # Import the wavfile API
import numpy as np

def get_samples(filepath):
    fs, data = wavfile.read(filepath)  # Load the data
    if len(data.shape) > 1:  # Check if stereo
        data = data.mean(axis=1)  # Convert to mono by averaging channels
    return data, fs

def fourier_transform(samples):
    spectrum = fft(samples)
    half = len(spectrum) // 2  # Take the first half of the spectrum
    return spectrum[:half]

def get_peak_frequency(spectrum, samplerate):
    # Get the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    # Find the index of the peak frequency
    peak_idx = np.argmax(magnitude)
    # Map the index to a frequency value
    frequency = peak_idx * (samplerate / (2 * len(spectrum)))
    return frequency

def get_frequency_from_file(filepath):
    samples, samplerate = get_samples(filepath)
    spectrum = fourier_transform(samples)
    return get_peak_frequency(spectrum, samplerate)

#### Cent Messgerät
Für Musiker ist es üblich die Frequenzdifferenz in Cent zu messen.

In [3]:
def frequency_difference_to_cent(f1,f2):
    return 1200 * np.log2(f1/f2) # 1200 Cent entsprechen einem Halbton


#### Visualisierungssoftware

Die Visualisierung zeigt die Frequenz der Saiten in Abhängigkeit von der Verstimmung.
Die Daten zum Visualisieren werden wie folgt strukturiert:

```
{
    'E2': {
        'E2': array([-4,...,4]),
        'D3': array([record_-4,record_-3,...,record_4]),
        ...,
        'E4': array([records_per_step,...])
    },
    ...
    'E4': {
        'E2': array([records_per_step,...]),
        'A2': array([records_per_step,...]),
        ...,
        'E4': array([-4,...,4])
    }
}
```

Das folgende Programm ermöglicht es uns, die Daten zu visualisieren. Es wird eine Checkbox für jede Saite erstellt, die es uns ermöglicht, die Daten für jede Saite einzeln anzuzeigen. Da interessant ist, ob die Verstimmung Logarithmisch ist, wird die änderung er Frequenz in Cent und in Hz dargestellt.

In [4]:
import matplotlib.pyplot as plt
from ipywidgets import  HBox, VBox, interactive, Layout, Checkbox, fixed
import random

nStrings = 6
steps = 4
step_range = range(-steps, steps+1) # -10 bis 10
strings = ["E2", "A2", "D3", "G3", "B3", "E4"]
# Verstimmung jeder anderen seite für jede Saite für alle Verstimmungsschritte
example_dataset = { # Dummy Data
    string: {other_string: np.array([400+( -_*random.random()if other_string != string else _ )for _ in step_range]) for other_string in strings}
    for string in strings
}


def visualisation(df,string,label,**args):
    for other_string in strings:
        if other_string in args.keys() and args[other_string]:
            ## Lineare Regression
            x = df[string][string]
            y = df[string][other_string]
            #m,b = np.polyfit(x, y, 1)
            #plt.plot(x, m*x+b, label=f"Linear Fit {m}x+{b} {other_string}")
            plt.plot(df[string][string], df[string][other_string], label=other_string)

    plt.title("Impact on String when detuning String "+string)
    plt.xlabel(f"Detuning of String {string} in {label}")
    plt.ylabel(f"Frequency of other Strings in {label}")

    plt.legend()


def visualize_all(data,label):
    widget_list = []

    for string in strings:
        checkboxes = {string: Checkbox(value=True,label=string,indent=False) for string in strings}
        widget = interactive(visualisation,df=fixed(data),string=fixed(string),label=fixed(label),**checkboxes)
        controls = HBox(widget.children[:-1]) # Horizontale Box für die Checkboxes
        output = widget.children[-1]
        w=VBox([controls, output], layout=Layout(margin="10px"))
        widget_list.append(w)
    row1 = HBox(widget_list[:3])
    row2 = HBox(widget_list[3:])
    output = VBox([row1, row2])
    display(output)



visualize_all(example_dataset,label="Eier")


### Durchführung

Die Audio-Dateien sind im Ordner `audio` zu finden.
Die Ordnerstruktur ist wie folgt:

    ```
    audio/<variable_saite>/<beeinflusste_saite>/<verstimmungsschritt>.wav

    audio
    ├── E2 # Ändernde Saite
      ├── E2 # Beinflusste Saite
        ├─── -4.wav
        ├─── ...
        ├─── 4.wav
      ├─── ...
      ├─── E4
    ├── ...
    ├── E4

#### Setup

<div class="imagestyle">
    <img src="assets/setup.jpg" width="500"/>
    Die Gitarre wird per Klinken Kabel an eine Audio Karte angeschlossen und mit einer Abtastrate von 44100 Hz aufgenommen.
Das Bild Zeigt wie die Gitarre positioniert war, damit der Gitarrenhals sich nicht verzieht und dann den Ton beinflusst.

</div>


<div class="imagestyle">
    <img src="assets/digital_setup.png" width="500"/>
    Es wird die Audio Workstation "Cubase" verwendet um die Samples Aufzunehmen. Die Aufnahme erfolgt in Mono. Die Aufnahme wird in 16 Bit und 44100 Hz aufgenommen.
    Es wird ein integriertes Stimmgerät verwendet, um die Saiten wieder zurück zu stimmen.
Jeder Aufnahmeblock enthält den Klang aller Saiten der Gitarre. Also klingend E2, A2, D3, G3, B3 und E4. Die Saiten werden in der Reihenfolge E2, A2, D3, G3, B3 und E4 gespielt.

Die Farbe der Aufnahme Blöcke ist eine Visualisierung dafür wie stark die Saiten verstimmt wurden und in welche Richtung. Je röter die Farbe, desto stärker die Verstimmung zu tieferen Frequenzen. Je blauer die Farbe, desto stärker die Verstimmung zu höheren Frequenzen.

Jede Zeile widmet sich einer Saite und einer Verstimmungsrichtung. Von oben nach unten ist die Reihenfolge: E2 nach oben, E2 nach unten, A2 nach oben, A2 nach unten, ..., E4 nach oben, E4 nach unten.
</div>


<div class="imagestyle">
<img src="assets/cutted_audio.png" width="500"/>
Anschließen werden die Einzelnen Audioblöcke so zu geschnitten, dass der Transient und das Verstummen der Saite verschwindet.
</div>


### Clean Up

<div class="imagestyle">
    <img src="assets/need_to_clean_data.png" width="500"/>
    Der Screenshot Zeigt, dass nicht die Grundfrequenz als solche erkannt wird, sondern teilweise Obertöne als die Grundfrequenz erkannt werden. <br>
    Deshalb wird für jede Saite ein Bandpass Filter erstellt, der die Obertöne reduziert.
</div>

Banpass Filter:
<div class="imagestyle">
    <img src="assets/bandpass_e2.png" width="500"/>
    Bandpass für E2
<div>
<div class="imagestyle">
    <img src="assets/bandpass_a2.png" width="500"/>
    Bandpass für A2
</div class="imagestyle">
<div class="imagestyle">
    <img src="assets/bandpass_d3.png" width="500"/>
    Bandpass für D3
</div>
<div class="imagestyle">
    <img src="assets/bandpass_g3.png" width="500"/>
    Bandpass für G3
</div>
<div class="imagestyle">
    <img src="assets/bandpass_b3.png" width="500"/>
    Bandpass für B3
</div>
<div class="imagestyle">
    <img src="assets/bandpass_e4.png" width="500"/>
    Bandpass für E4
</div>

### Laden der Daten


In [5]:
# Experiment
measured_data = {}
filenames =  [f"{i}.wav" for i in step_range]

for changing_string in strings:
    for impacted_string in strings:
        #f0 = get_frequency_from_file(f"audio/{changing_string}/{impacted_string}/0.wav")

        for filename in filenames:
            frequency = get_frequency_from_file(f"audio/{changing_string}/{impacted_string}/{filename}")

            print(f"Saite: {changing_string}, Beeinflusste Saite: {impacted_string}, Verstimmungsschritt: {filename}, Frequenz: {frequency}")

            if changing_string not in measured_data:
                measured_data[changing_string] = {}

            if impacted_string not in measured_data[changing_string]:
                measured_data[changing_string][impacted_string] = []

            measured_data[changing_string][impacted_string].append(frequency)


C:\Users\guita\AppData\Local\Temp\ipykernel_4024\3270566540.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, data = wavfile.read(filepath)  # Load the data


Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -4.wav, Frequenz: 57.56155443488579
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -3.wav, Frequenz: 67.89055643973762
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -2.wav, Frequenz: 74.80436177036562
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: -1.wav, Frequenz: 78.13278008298755
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 0.wav, Frequenz: 82.29854689564068
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 1.wav, Frequenz: 84.9931433230792
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 2.wav, Frequenz: 87.45281541913823
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 3.wav, Frequenz: 92.93188805334641
Saite: E2, Beeinflusste Saite: E2, Verstimmungsschritt: 4.wav, Frequenz: 97.02425718679493
Saite: E2, Beeinflusste Saite: A2, Verstimmungsschritt: -4.wav, Frequenz: 114.39652438637661
Saite: E2, Beeinflusste Saite: A2, Verstimmungsschritt: -3.wav, Frequenz: 112.8823800

#### Aggregating Data
Gemessen haben wir die Tatsächlichen Frequenzen und wie Sie sich Inabhängigkeit jeder anderen Saite ändert. Jedoch interessiert uns vorallem das Maß dieser Änderung. Daher werden wir jeden Messwert zu seinem Ausgangswert in Cent und Herz umwandeln.


In [6]:
cent_changes = { # Dummy Data
    string: {other_string: np.array([float(0) for _ in step_range]) for other_string in strings}
    for string in strings
}
hz_changes = cent_changes.copy()

for changing_string in strings:
    for impacted_string in strings:
        inital_value = measured_data[changing_string][impacted_string][steps]
        for i in range(steps*2+1): # -4 bis 4
            cent_changes[changing_string][impacted_string][i] = frequency_difference_to_cent(measured_data[changing_string][impacted_string][i], inital_value)
            hz_changes[changing_string][impacted_string][i] = measured_data[changing_string][impacted_string][i] - inital_value


## 3. Visualisierung

In [7]:
print("Frequenz in Abhängigkeit der Frequenz")
visualize_all(measured_data,label="Hz")
print("Frequenzänderung in Cent")
visualize_all(cent_changes,label="Cent")
print("Frequenzänderung in Hz")
visualize_all(hz_changes,label="Hz")

Frequenz in Abhängigkeit der Frequenz


Frequenzänderung in Cent


Frequenzänderung in Hz



### Elastizität und Linearität
Es ist elastisch, weil ausgang und endfrequenzen gleich sind. Es ist nicht perfekt linear, aber dafür hinreichend Linear, da wir mit kleinen Wertenänderung arbeiten.
 ...
### 4.1 Mathematisches Modell

Wir können das System als ein lineares Gleichungssystem beschreiben. Die Frequenz der Saiten kann als Vektor dargestellt werden. Die Verstimmung der Saiten kann als Matrix dargestellt werden. Die Verstimmung der Saiten ist eine lineare Transformation der Frequenz der Saiten.

$$
\vec{f_0} = \begin{pmatrix} {f_{E2}} \\ {f_{A2}} \\ {f_{D3}} \\ {f_{G3}} \\ {f_{B3}} \\ {f_{e4}} \end{pmatrix},
C = \begin{bmatrix} {1} & { c_{12} } & {c_{13}} & {c_{14}} & {c_{15}} & {c_{16}} \\ {c_{21}} & {1} & {c_{23}} & {c_{24}} & {c_{25}} & {c_{26}} \\ {c_{31}} & {c_{32}} & {1} & {c_{34}} & {c_{35}} & {c_{36}} \\ {c_{41}} & {c_{42}} & {c_{43}} & {1} & {c_{45}} & {c_{46}} \\ {c_{51}} & {c_{52}} & {c_{53}} & {c_{54}} & {1} & {c_{56}} \\ {c_{61}} & {c_{62}} & {c_{63}} & {c_{64}} & {c_{65}} & {1} \end{bmatrix},
\vec{g} = \begin{pmatrix} {\hat{f}_{E2}} \\ {\hat{f}_{A2}} \\ {\hat{f}_{D3}} \\ {\hat{f}_{G3}} \\ {\hat{f}_{B3}} \\ {\hat{f}_{e4}} \end{pmatrix}
$$
\
$\vec{f_0}$ sind die Frequenzen der Saiten in der Ausgangsposition.\
$C$ ist die Verstimmungsmatrix. Die Elemente $c_{ij}$ sind die Verstimmungsfaktoren der Saite $i$, wenn die Saite $j$ um 1 Schritt (20 Cent) verstimmt wird.\
$\vec{g}$ sind die Frequenzen der Saiten nach der Verstimmung.\
$\vec{g}$ wollen wir im allgemeinen als Zielfrequenzen erreichen.

Die Annahme ist, es gibt eine Verstimmung $\vec{\delta}$ der Saiten, die die Frequenzen der Saiten in $\vec{g}$ erreicht, wenn sie dem System $C$ unterworfen wird.

$$
\vec{g} = add\_cent\_to\_freq(C \cdot \vec{\delta}, \vec{f_0})
$$

$$
\vec{\delta} = cent(\vec{f_0}-\vec{g}) \cdot C^{-1}
$$

$C^{-1}$ ist die Inverse der Verstimmungsmatrix.\
$cent$ ist die Funktion, die die Frequenzdifferenz in Cent umwandelt.\
$add\_cent\_to\_freq$ ist die Funktion, die die Verstimmung in Cent zu den Frequenzen der Saiten addiert.

### 5. Programmierung
Im folgendem werden die Funktionen `cent`, `add_cent_to_freq` und `find_coefficients` implementiert.

#### 5.1 Funktionen




In [8]:
def cent(f1,f2):
    return 1200 * np.log2(f1/f2) # 1200 Cent entsprechen einer Oktave

def add_cent_to_freq(cent,freq):
    return freq * 2 ** (cent/1200)/12

def find_coefficients(f0, g, C):
    return (g-f0) @ np.linalg.inv(C)


## Literatur
[1]Gleich temperierte Stimmung: https://de.wikipedia.org/wiki/Gleichstufige_Stimmung \
[2]Cent: https://de.wikipedia.org/wiki/Cent_(Musik) \
[3]Vibration of a String: https://en.wikipedia.org/wiki/String_vibration \
[4]Mechanische Spannung: https://de.wikipedia.org/wiki/Mechanische_Spannung \
[5]Hookesches Gesetz: https://de.wikipedia.org/wiki/Hookesches_Gesetz

## TODO
- Einfluss vom Radius u
- reale Wertebereiche für das Modell Nutzen $\beta$ usw...
- scipy peak detection
-